In [ ]:
import pandas as pd
import re
!pip install tqdm
from tqdm import tqdm
# use tqdm with pandas
# use progress_apply instead of normal apply
tqdm.pandas()
import string
!pip install plotly
pd.options.plotting.backend = "plotly"
import plotly.io as pio
import plotly.express as px
pio.templates.default = "plotly_white"

/opt/venv/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [ ]:
# vectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# splitting and evaluation of model
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
# NLTK stuff
!pip install nltk
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.stem.snowball import EnglishStemmer
from nltk import word_tokenize

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Models:
- Logistic Reg
- SVM
- KNN

Let's import the data

In [ ]:
sample_submission = pd.read_csv("/datasets/real-or-not-dataset/sample_submission.csv")
test = pd.read_csv("/datasets/real-or-not-dataset/test.csv")
train = pd.read_csv("/datasets/real-or-not-dataset/train.csv")

In [ ]:
n = 10
random_state = 3
modified = train

In [ ]:
stop_w = stopwords.words('english')
stemmer = EnglishStemmer(ignore_stopwords=True)
!pip install flashgeotext
# for finding the city names properly
from flashgeotext.geotext import GeoText
geotext = GeoText(use_demo_data=True)

2020-06-24 19:59:34.279 | DEBUG    | flashgeotext.lookup:add:194 - cities added to pool
2020-06-24 19:59:34.284 | DEBUG    | flashgeotext.lookup:add:194 - countries added to pool
2020-06-24 19:59:34.285 | DEBUG    | flashgeotext.lookup:_add_demo_data:225 - demo data loaded for: ['cities', 'countries']


Process the Cols

In [ ]:
# Process non null keywords and locations
modified["keyword"].fillna("", inplace = True)
modified["location"].fillna("", inplace = True)
modified.sample(n=n,random_state=random_state)

,id,keyword,location,text,target
1506,2174,catastrophic,Inexpressible Island,The Catastrophic Effects of Hiroshima and Naga...,1
5763,8225,riot,Los Angeles,@eac4AU You can now PRE-ORDER the film on ITUN...,0
5157,7356,obliterate,Purfleet,Whereas Jez will obliterate the national debt ...,1
4550,6467,injured,,Washington Post - 4 dead dozens injured in Gaz...,1
4329,6148,hijack,Nigeria,Criminals Who Hijack Lorries And Buses Arreste...,1
1420,2049,casualties,"Afghanistan, USA",#Afghanistan: sharp rise in women and children...,1
6984,10016,twister,Detroit,Crazy Mom Threw Teen Daughter a NUDE Twister S...,0
7541,10782,wreckage,"New Delhi,India",Wreckage 'Conclusively Confirmed' as From MH37...,1
979,1418,body%20bag,New York,genuine Leather man Bag Messenger fit iPad min...,0
6671,9562,thunder,"Atlanta, GA",Am I hearing thunder or trucks?,0


In [ ]:
def makecountryarray(x):
    countries = geotext.extract(input_text=str(x), span_info=False)['countries']
    country = ""
    for items in countries:
        country =  country + ' ' + items
    # print("location : {}, got {}".format(x, country))
    return country
print(makecountryarray("I'd like to build a wall between Mexico and Usa"))

 Mexico United States


In [ ]:
# Convert the weird location to a more standardized tag
modified.location = modified.location.progress_apply(makecountryarray)

100%|██████████| 7613/7613 [00:00<00:00, 97500.85it/s]


In [ ]:
modified.sample(n=n,random_state=random_state)

,id,keyword,location,text,target
1506,2174,catastrophic,,The Catastrophic Effects of Hiroshima and Naga...,1
5763,8225,riot,,@eac4AU You can now PRE-ORDER the film on ITUN...,0
5157,7356,obliterate,,Whereas Jez will obliterate the national debt ...,1
4550,6467,injured,,Washington Post - 4 dead dozens injured in Gaz...,1
4329,6148,hijack,,Criminals Who Hijack Lorries And Buses Arreste...,1
1420,2049,casualties,Afghanistan United States,#Afghanistan: sharp rise in women and children...,1
6984,10016,twister,,Crazy Mom Threw Teen Daughter a NUDE Twister S...,0
7541,10782,wreckage,India,Wreckage 'Conclusively Confirmed' as From MH37...,1
979,1418,body%20bag,,genuine Leather man Bag Messenger fit iPad min...,0
6671,9562,thunder,,Am I hearing thunder or trucks?,0


In [ ]:
def vectandstem(x):
    tokenized = word_tokenize(str(x))

    full= " ".join(tokenized)
    return full

print(vectandstem("emergencies totally"))

emergencies totally


In [ ]:
# Vectorize and stem the keywords
modified.keyword = modified.keyword.progress_apply(vectandstem)

100%|██████████| 7613/7613 [00:00<00:00, 11049.78it/s]


In [ ]:
modified.sample(n=n,random_state=random_state)

,id,keyword,location,text,target
1506,2174,catastrophic,,The Catastrophic Effects of Hiroshima and Naga...,1
5763,8225,riot,,@eac4AU You can now PRE-ORDER the film on ITUN...,0
5157,7356,obliterate,,Whereas Jez will obliterate the national debt ...,1
4550,6467,injured,,Washington Post - 4 dead dozens injured in Gaz...,1
4329,6148,hijack,,Criminals Who Hijack Lorries And Buses Arreste...,1
1420,2049,casualties,Afghanistan United States,#Afghanistan: sharp rise in women and children...,1
6984,10016,twister,,Crazy Mom Threw Teen Daughter a NUDE Twister S...,0
7541,10782,wreckage,India,Wreckage 'Conclusively Confirmed' as From MH37...,1
979,1418,body % 20bag,,genuine Leather man Bag Messenger fit iPad min...,0
6671,9562,thunder,,Am I hearing thunder or trucks?,0


In [ ]:
#Function to Vectorize and clean twes 
def hiICleanTrash(x):
    # first pass to clean the punctuation
    pass1  = "".join([char for char in x if char not in string.punctuation])
    # second pass to clean the numbers up
    pass2 = re.sub('[0-9]+', '', pass1)
    # third pass to tokenize
    pass3 = word_tokenize(pass2)
    # fourth pass to use snowball to stem words if they're not a stopword
    pass4 = [stemmer.stem(word) for word in pass3 if word not in stop_w]
    return str(pass4)
print(hiICleanTrash("Hello! I like cleaning the trash. I found 4 occurances today!"))

['hello', 'i', 'like', 'clean', 'trash', 'i', 'found', 'occur', 'today']


In [ ]:
modified.text = modified.text.progress_apply(hiICleanTrash)

100%|██████████| 7613/7613 [00:02<00:00, 2629.91it/s]


In [ ]:
modified.sample(n=n,random_state=random_state)

,id,keyword,location,text,target
1506,2174,catastrophic,,"['the', 'catastroph', 'effect', 'hiroshima', '...",1
5763,8225,riot,,"['eacau', 'you', 'preorder', 'film', 'itun', '...",0
5157,7356,obliterate,,"['wherea', 'jez', 'obliter', 'nation', 'debt',...",1
4550,6467,injured,,"['washington', 'post', 'dead', 'dozen', 'injur...",1
4329,6148,hijack,,"['crimin', 'who', 'hijack', 'lorri', 'and', 'b...",1
1420,2049,casualties,Afghanistan United States,"['afghanistan', 'sharp', 'rise', 'women', 'chi...",1
6984,10016,twister,,"['crazi', 'mom', 'threw', 'teen', 'daughter', ...",0
7541,10782,wreckage,India,"['wreckag', 'conclus', 'confirm', 'from', 'mh'...",1
979,1418,body % 20bag,,"['genuin', 'leather', 'man', 'bag', 'messeng',...",0
6671,9562,thunder,,"['am', 'i', 'hear', 'thunder', 'truck']",0


In [ ]:
# Create a new df
model_df = pd.DataFrame()

In [ ]:
def get_info():
    fig = px.box(model_df.melt(var_name="Model"),
        y="value",labels={
            'value':'Accuracy'
        },
        facet_col="Model", color="Model",
        boxmode="overlay",
        points='all')
    # Remove the Model = **
    fig.for_each_annotation(lambda a: a.update(text=''))
    fig.show()

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
def logisticreg(X,Y,CV,SLVR,PNLTY,VRB):
  X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2,random_state=0)
  
  vectorizer = CountVectorizer(stop_words='english')
  
  Xtrain_vectors = vectorizer.fit_transform(X_train)
  test_vectors = vectorizer.transform(X_test)
  
  tfidf = TfidfTransformer()
  
  Xtrain_vectors = tfidf.fit_transform(Xtrain_vectors)
  test_vectors = tfidf.transform(test_vectors)

  feature = vectorizer.fit_transform(X)
  label = Y
  
  clf = LogisticRegression(C=1, solver=SLVR, penalty=PNLTY,max_iter=700, verbose=VRB)
  clf.fit(Xtrain_vectors,y_train)

  accuracies = cross_val_score(clf,feature, label, scoring='accuracy', cv=CV)

  return accuracies

In [ ]:
model_df['Logistic Regression liblinear+l1'] = logisticreg(modified["text"],train["target"],10,'liblinear','l1',0)
print("Done with 1st")
model_df['Logistic Regression lbfgs+l2'] = logisticreg(modified["text"],train["target"],10,'lbfgs','l2',0)

get_info()

Done with 1st


In [ ]:
from sklearn.svm import SVC
from sklearn.svm import LinearSVC

In [ ]:
def support_vectors(X, Y, CV, KNL='rbf', VRB=False, DG=3):
    X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2,random_state=0)

    vectorizer = CountVectorizer(stop_words='english')

    Xtrain_vectors = vectorizer.fit_transform(X_train)
    test_vectors = vectorizer.transform(X_test)

    tfidf = TfidfTransformer()

    Xtrain_vectors = tfidf.fit_transform(Xtrain_vectors)
    test_vectors = tfidf.transform(test_vectors)

    feature = vectorizer.fit_transform(X)
    label = Y
    if(KNL=='rbf'):
        clf = SVC(C=1, verbose=VRB, gamma=.01, kernel=KNL, degree=DG)
    elif (KNL=='linear'):
        clf = LinearSVC(C=1, verbose=VRB)
    else:
        raise TypeError('Only rbf and linear types are allowed')
    clf.fit(Xtrain_vectors,y_train)

    accuracies = cross_val_score(clf,feature, label, scoring='accuracy', cv=CV)

    return accuracies

In [ ]:
model_df['SVM rbf+3°'] = support_vectors(modified["text"], train["target"], 10,'rbf',1,3)
print("Done with 1st")
model_df['SVM linear+2°'] = support_vectors(modified["text"], train["target"], 10,'linear',1)

get_info()

[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]Done with 1st
[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
def naive_bayes(X,Y,CV):
  X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2,random_state=0)
  
  vectorizer = CountVectorizer(stop_words='english')
  
  Xtrain_vectors = vectorizer.fit_transform(X_train)
  test_vectors = vectorizer.transform(X_test)
  
  tfidf = TfidfTransformer()
  
  Xtrain_vectors = tfidf.fit_transform(Xtrain_vectors)
  test_vectors = tfidf.transform(test_vectors)

  feature = vectorizer.fit_transform(X)
  label = Y
  
  clf = MultinomialNB()
  clf.fit(Xtrain_vectors,y_train)

  accuracies = cross_val_score(clf,feature, label, scoring='accuracy', cv=CV)

  return accuracies

In [ ]:
model_df['NaiveBayes MultiNomial'] = naive_bayes(modified["text"], train["target"], 10)

get_info()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
def knn_classifier(X,Y,CV,NBR):
  X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
  
  vectorizer = CountVectorizer(stop_words='english')
  
  Xtrain_vectors = vectorizer.fit_transform(X_train)
  test_vectors = vectorizer.transform(X_test)
  
  tfidf = TfidfTransformer()
  
  Xtrain_vectors = tfidf.fit_transform(Xtrain_vectors)
  test_vectors = tfidf.transform(test_vectors)

  feature = vectorizer.fit_transform(X)
  label = Y
  
  clf = KNeighborsClassifier(NBR)
  clf.fit(Xtrain_vectors, y_train)

  accuracies = cross_val_score(clf, feature, label, scoring='accuracy', cv=CV)

  return accuracies

In [ ]:
model_df['KNN 1N'] = knn_classifier(modified["text"], train["target"], 10, 1)
print("Done with 1st")
model_df['KNN 5N'] = knn_classifier(modified["text"], train["target"], 10, 5)

get_info()

Done with 1st


KNN is prolly not the best thing for this task